In [3]:
!pip install -q opencv-python matplotlib


In [5]:
import os, cv2, csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:

from cv2 import GaussianBlur

# Generates a RGB histogram for a single image
def plot_rgb_histogram(image_path):
    #image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converti da BGR a RGB
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    hist_gray = cv2.calcHist([image], [0], None, [256], [0, 256])
    
    ''' RESIZING
    # Set rows and columns 
    # lets downsize the image using new  width and height
    down_width = 300
    down_height = 200
    down_points = (down_width, down_height)
    resize_down = cv2.resize(image, down_points, interpolation= cv2.INTER_LINEAR)
    # Display images
    cv2.imshow('Resized Down by defining height and width', resized_down)
    cv2.waitKey()
    cv2.imshow('Resized Up image by defining height and width', resized_up)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
    
    ##################################################################### GAUSSIAN BLUR AND LAPLACIAN
    ddepth = cv2.CV_16S
    kernel_size = 3
    window_name = "Laplace Demo"
    # [variables]
    # [load]
    imageName = "dataset/distanzeProva/PULITE.png"
    src = cv2.imread(cv2.samples.findFile(imageName), cv2.IMREAD_COLOR) # Load an image
    # Check if image is loaded fine
    if src is None:
        print ('Error opening image')
        print ('Program Arguments: [image_name -- default lena.jpg]')
        return -1
    # [load]
    src = cv2.GaussianBlur(src, (3, 3), 0)
    # [reduce_noise]
    # [convert_to_gray]
    # Convert the image to grayscale
    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    # [convert_to_gray]
    # Create Window
    cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)
    # [laplacian]
    # Apply Laplace function
    dst = cv2.Laplacian(src_gray, ddepth, ksize=kernel_size)
    # [laplacian]
    # [convert]
    # converting back to uint8
    abs_dst = cv2.convertScaleAbs(dst)
    # [convert]
    # [display]
    cv2.imshow(window_name, abs_dst)
    cv2.waitKey(0)
    ####################################################################
    '''
    plt.figure()
    plt.title(f'Istogramma di {os.path.basename(image_path)}')
    plt.xlabel('Intensità di colore')
    plt.ylabel('Numero di pixel')
    
    plt.plot(hist_gray, color='r')
    
    plt.xlim([0, 256])
    plt.legend()
    #plt.show()
    return hist_gray

def process_images_from_folder(folder_path):
    if not os.path.exists(folder_path):
        print("Error: Specified folder does not exists.")
        return
    
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg', 'webp'))]
    if not image_files:
        print("No images found in the folder.")
        return
    
    histograms_list = []
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        hist = plot_rgb_histogram(image_path)  
        histograms_list.append((image_file, hist))
    
    distance_matrix = pd.DataFrame(0, index=image_files, columns=image_files, dtype='float64')

    # Calculating distance between images histograms
    for i in range(len(histograms_list)):
        for j in range(i, len(histograms_list)):
            image1, hist1 = histograms_list[i]
            image2, hist2 = histograms_list[j]
            
            total_distance = 0
            total_distance +=  cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
            total_distance = 1 - total_distance # otherwise total_distance is similarity

            distance_matrix.loc[image1, image2] = total_distance #distance values
            distance_matrix.loc[image2, image1] = total_distance
            
    distance_matrix.to_csv(os.path.join(folder_path,"distance_matrix.csv"))

folder_path = "datasetSuddiviso/UV1"
process_images_from_folder(folder_path)

TypeError: Can't convert object to 'str' for 'relative_path'